# DICE Basic Results

In [1]:
import sys
import os
sys.path.append(os.path.join(os.getcwd(), '../../..'))

%load_ext autoreload
%autoreload 2

import numba
import pandas as pd
import numpy as np
from sklearn import neighbors
from sklearn import model_selection

import matplotlib.pyplot as plt
import seaborn as sns
from models import model_interface, model_loader, model_constants
from data import data_loader
from data.adapters import continuous_adapter

# Preliminaries -- load everything

In [3]:
DATASET, DATASET_INFO = data_loader.load_data(data_loader.DatasetName('credit_card_default'))
MODEL = model_loader.load_model(model_constants.ModelType('logistic_regression'), data_loader.DatasetName('credit_card_default'))
ADAPTER = continuous_adapter.StandardizingAdapter(
    label_column = DATASET_INFO.label_column, positive_label=DATASET_INFO.positive_label
).fit(DATASET)

results_dir = '../../../experiment_results/dice_results/dice_comparison'

index_df = pd.read_csv(os.path.join(results_dir, 'experiment_config_df.csv'))
path_df = pd.read_csv(os.path.join(results_dir, 'dice_paths_df.csv'))
index_df

/home/jakeval/umass/research/.venv/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator StandardScaler from version 1.1.2 when using version 1.1.3. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/jakeval/umass/research/.venv/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator LogisticRegression from version 1.1.2 when using version 1.1.3. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


,batch_id,run_id,run_seed,confidence_cutoff,dataset_name,max_iterations,model_type,noise_ratio,num_paths,rescale_ratio
0,6,202,171,0.7,credit_card_default,30,logistic_regression,NaN,4,NaN
1,1,32,1701,0.6,credit_card_default,30,logistic_regression,NaN,3,NaN
2,10,321,9251,0.8,credit_card_default,30,logistic_regression,NaN,4,NaN
3,5,176,7142,0.7,credit_card_default,30,logistic_regression,NaN,3,NaN
4,8,257,1573,0.8,credit_card_default,30,logistic_regression,NaN,2,NaN
...,...,...,...,...,...,...,...,...,...,...
475,13,391,3183,0.9,credit_card_default,30,logistic_regression,NaN,3,NaN
476,7,215,8286,0.7,credit_card_default,30,logistic_regression,NaN,5,NaN
477,8,263,211,0.8,credit_card_default,30,logistic_regression,NaN,2,NaN
478,15,450,7527,0.9,credit_card_default,30,logistic_regression,NaN,5,NaN


## Fit a KDE

In [6]:
kfold = model_selection.KFold(n_splits=5)
transformed_data = ADAPTER.transform(DATASET.drop(columns='Y')).sample(frac=1, replace=False)

bandwidths = np.logspace(-1, 0, 6)
scores = []

for bw in bandwidths:
    print("Evaluate bandwidth", bw)
    score = 0
    for train_indices, val_indices in kfold.split(transformed_data):
        kde = neighbors.KernelDensity(bandwidth=bw).fit(transformed_data.iloc[train_indices])
        score += kde.score(transformed_data.iloc[val_indices])
    scores.append(score / len(bandwidths))
    print(scores[-1])
best_bandwidth = bandwidths[np.argmax(scores)]
print("Finished! Selected bandwidth is ", best_bandwidth)
KDE = neighbors.KernelDensity(bandwidth=bw).fit(transformed_data)
print("Scoring full dataset...")
scores = KDE.score_samples(transformed_data)

Evaluate bandwidth 0.1


KeyboardInterrupt: 

### Evaluate the KDE qualitatively

The scores are between -28 and -20. Unsurprisingly, most points have relatively high density.
Overall the histogram seems reasonable.

In [4]:
#best_bandwidth = bandwidths[np.argmax(scores)]
transformed_data = ADAPTER.transform(DATASET.drop(columns='Y')).sample(frac=1, replace=False)
best_bandwidth = 0.251188643150958
KDE = neighbors.KernelDensity(bandwidth=best_bandwidth).fit(transformed_data)
#KDE_SCORES = KDE.score_samples(transformed_data)
#sns.histplot(KDE_SCORES)
#pd.DataFrame({'density': KDE_SCORES}).describe()

# Analyze the results

In [5]:
# Generate some numpy arrays so we can pass of to numba
# The slowest here will be KDE

# order the paths dataframe
ordered_paths = path_df.sort_values(['run_id', 'path_id', 'step_id'])
run_ids = ordered_paths.run_id.to_numpy()
path_ids = ordered_paths.path_id.to_numpy()

# get the raw data -- we've already extracted the run_id and path_id.
paths = ordered_paths.drop(columns=['run_id', 'batch_id', 'step_id', 'path_id'])

# pos_proba and target_proba are calculated for every point in every path
pos_proba = MODEL.predict_pos_proba(paths).to_numpy()
target_proba = ordered_paths.merge(index_df[['run_id', 'confidence_cutoff']], how='left', on='run_id', validate='many_to_one').confidence_cutoff.to_numpy()

numpy_paths = ADAPTER.transform(paths).to_numpy()

# the first path begins at boundary_indices[0]. The second path begins at boundary_indices[1]. There is no path beginning at boundary_indices[-1].
boundary_indices = np.arange(run_ids.shape[0])[(path_ids != np.hstack([[-1], path_ids[:-1]]))]
boundary_indices = np.hstack([boundary_indices, path_ids.shape[0]])

# run KDE over the POIs and CFEs
pois = paths.iloc[boundary_indices[:-1]]
counterfactuals = paths.iloc[boundary_indices[1:] - 1]
poi_kde = KDE.score_samples(ADAPTER.transform(pois))
cfe_kde = KDE.score_samples(ADAPTER.transform(counterfactuals))

In [6]:
@numba.jit(nopython=True)
def get_sparsity(path: np.ndarray) -> int:
    if path.shape[0] == 1:
        return np.nan
    path_sparsity = np.zeros(path.shape[0])
    for i in range(1, path.shape[0]):
        path_sparsity[i] = ((path[i] - path[i - 1]) != 0).sum()
    return np.max(path_sparsity)

@numba.jit(nopython=True)
def get_path_length(path: np.ndarray) -> float:
    total = 0
    for i in range(1, path.shape[0]):
        total += np.linalg.norm(path[i] - path[i - 1])
    if total == 0:
        return np.nan
    return total

@numba.jit(nopython=True)
def analyze_paths(
    paths: np.ndarray,
    run_ids: np.ndarray,
    path_ids: np.ndarray,
    pos_proba: np.ndarray,
    target_proba: np.ndarray,
    boundary_indices: np.ndarray,
    poi_kde: np.ndarray,
    cfe_kde: np.ndarray
):
    columns = ['run_id', 'path_id', 'success', 'path_length', 'poi_density', 'cfe_density', 'sparsity']
    col_idx = {}
    for i, col in enumerate(columns):
        col_idx[col] = i
    results = np.zeros((len(boundary_indices) - 1, len(columns)))
    for i in range(boundary_indices.shape[0]-1):
        start_idx, end_idx = boundary_indices[i:i+2]
        path = paths[start_idx:end_idx]
        results[i,col_idx['run_id']] = run_ids[start_idx]
        results[i,col_idx['path_id']] = path_ids[start_idx]
        results[i,col_idx['success']] = 1 if pos_proba[end_idx - 1] >= target_proba[end_idx - 1] else 0
        results[i,col_idx['path_length']] = get_path_length(path)
        results[i,col_idx['poi_density']] = poi_kde[i]
        results[i,col_idx['cfe_density']] = cfe_kde[i]
        results[i,col_idx['sparsity']] = get_sparsity(path)

    return results, columns

In [9]:
numpy_results, columns = analyze_paths(numpy_paths, run_ids, path_ids, pos_proba, target_proba, boundary_indices, poi_kde, cfe_kde)

results = pd.DataFrame(data=numpy_results, columns=columns).merge(index_df, how='left', on='run_id').drop(columns=['dataset_name', 'max_iterations', 'model_type', 'noise_ratio', 'rescale_ratio', 'run_seed'])
results

,run_id,path_id,success,path_length,poi_density,cfe_density,sparsity,batch_id,confidence_cutoff,num_paths
0,0.0,0.0,1.0,28.036184,-1.036496,-2452.813638,2.0,0,0.6,2
1,0.0,1.0,1.0,58.944101,-1.036496,-3740.954291,3.0,0,0.6,2
2,1.0,0.0,1.0,41.043119,-1.039537,-6381.968896,2.0,0,0.6,2
3,1.0,1.0,1.0,72.143734,-1.039537,-7744.260791,2.0,0,0.6,2
4,2.0,0.0,1.0,43.144045,3.340095,-7837.518642,1.0,0,0.6,2
...,...,...,...,...,...,...,...,...,...,...
1675,479.0,0.0,1.0,66.467965,-1.056702,-6263.579618,3.0,15,0.9,5
1676,479.0,1.0,1.0,45.859615,-1.056702,-10015.018235,2.0,15,0.9,5
1677,479.0,2.0,1.0,40.786306,-1.056702,-7505.067446,3.0,15,0.9,5
1678,479.0,3.0,1.0,63.488156,-1.056702,-4626.374966,2.0,15,0.9,5


# Let's look at the average metrics across the full batch


In [10]:
results.groupby('batch_id').mean().sort_values(['confidence_cutoff', 'num_paths'])

,run_id,path_id,success,path_length,poi_density,cfe_density,sparsity,confidence_cutoff,num_paths
batch_id,,,,,,,,,
0,14.5,0.5,1.0,38.023490,0.081015,-3450.251003,2.033333,0.6,2.0
1,44.5,1.0,1.0,35.101988,0.081015,-2987.608282,1.966667,0.6,3.0
2,74.5,1.5,1.0,34.283999,0.081015,-2923.994273,1.941667,0.6,4.0
3,104.5,2.0,1.0,33.268430,0.081015,-3019.493220,1.960000,0.6,5.0
4,134.5,0.5,1.0,39.764044,0.081015,-4221.555845,2.050000,0.7,2.0
5,164.5,1.0,1.0,38.715193,0.081015,-4062.557814,2.000000,0.7,3.0
6,194.5,1.5,1.0,38.863611,0.081015,-3888.310352,2.033333,0.7,4.0
7,224.5,2.0,1.0,37.976113,0.081015,-3662.685738,2.000000,0.7,5.0
8,254.5,0.5,1.0,40.232484,0.081015,-3185.040912,1.900000,0.8,2.0
